<a href="https://colab.research.google.com/github/Shreeshambav/DeepLearning_training/blob/main/EDA_MRL_LINE_Outliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##########################################################################################
############### File Merging - from different files
#########################################################################################
### Step1: Merging of files from multiple source

# import pandas as pd

# # List of Excel files to merge
# file_paths = [

#     'C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Rawfiles\\AI_2019_MGT_data.xls',

#     'C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Rawfiles\\AI_2020_MGT_data.xls',

#     'C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Rawfiles\\AI_2021_MGT_data.xls',

#     # Add more file paths as needed

# ]

# # Create an empty list to store DataFrames
# data_frames = []

# # Read each Excel file into a DataFrame and store in the list
# for file_path in file_paths:
#     df = pd.read_excel(file_path)
#     data_frames.append(df)

# # Create a new DataFrame to store the vertically merged data
# merged_df = pd.DataFrame()

# # Iterate through each DataFrame and merge vertically
# for df in data_frames:
#     year = df.iloc[0]["YEAR"]
#     df = df.drop(columns=["YEAR"])  # Remove the YEAR column to avoid duplication
#     month_columns = [col for col in df.columns if col.startswith("SUM(BALANCE_")]
#     df = df.rename(columns={col: col.replace("SUM(BALANCE_", "").replace(")", "") for col in month_columns})
#     df["Year"] = year
#     merged_df = pd.concat([merged_df, df], ignore_index=True)

# # Save the merged DataFrame to a new Excel file if needed
# merged_file_path = "C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Working_data\\Budget_merged_file_raw.xlsx"
# merged_df.to_excel(merged_file_path, index=False, engine="openpyxl")

# print("Files merged vertically successfully and saved to:", merged_file_path)


In [ ]:
##########################################################################################
############### Adding category column in merged file
#########################################################################################
# ### Step2: Categorisation based on given meta data file - MRL_LINE_NEW_GROUPING.xlsx
# import pandas as pd

# # Read input files
# file_path_budget = 'C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Working_data\\Budget_merged_file_raw.xlsx'
# file_path_category = 'C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Rawfiles\\MRL_LINE_NEW_GROUPING.xlsx'

# df_budget1 = pd.read_excel(file_path_budget)
# df_category = pd.read_excel(file_path_category)

# # Merge data based on 'MRL_LINE' column
# merged_df_cat = df_budget1.merge(df_category[['MRL_LINE', 'MRL_Category']], on='MRL_LINE', how='left')

# # Create the 'MRL_Category' column in the merged DataFrame
# df_budget1['MRL_Category'] = merged_df_cat['MRL_Category']

# # Write the updated DataFrame to the output file
# output_file_path = 'C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Working_data\\Budget_merged_file_Cat_working.xlsx'
# df_budget1.to_excel(output_file_path, index=False)

# print("Data merged and saved to:", output_file_path)

In [ ]:
##############################################################################################
#### Grouping data based on Category
##############################################################################################
## Step3: Feature engineering the data
import pandas as pd
# Read input files
file_path_merged = 'C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Working_data\\Budget_merged_file_Cat_working.xlsx'
df_merged_group_cat = pd.read_excel(file_path_merged)

# Problem statement: What should be the budget alloocated for next financial year.
# For further analysis we need to divide daatset into independent and dependent features
# MRL_Category is dependent column and rest are independent
# Following column do not add any value to analysis:
# Selection of Columns: LE_BOOK(2), OFFICE_ACCOUNT(3), VISION_SBU(5), ACCOUNT_OFFICER(6), PRODUCT(9)
print(df_merged_group_cat)

# Assuming df_merged_group_cat is the DataFrame containing the data
columns_to_drop = ['LE_BOOK', 'OFFICE_ACCOUNT', 'VISION_SBU', 'ACCOUNT_OFFICER', 'PRODUCT']
df_merged_group_cat = df_merged_group_cat.drop(columns=columns_to_drop)
df_merged_group_cat.shape

# Preparing Category for encodeing
category_mapping = {
    1: 'Cat1',
    2: 'Cat2',
    3: 'Cat3',
    4: 'Cat4'
}

df_merged_group_cat['MRL_Category'] = df_merged_group_cat['MRL_Category'].replace(category_mapping)

# Assuming df_merged_group_cat is the DataFrame containing the data
# One-hot encode the "MRL_Category" column
encoded_categories = pd.get_dummies(df_merged_group_cat["MRL_Category"], prefix="MRL_Category")

# Drop the original "MRL_Category" column
df_merged_group_cat.drop(columns=["MRL_Category"], inplace=True)

# Concatenate the encoded columns with the original DataFrame
df_merged_group_cat = pd.concat([df_merged_group_cat, encoded_categories], axis=1)
print(df_merged_group_cat)
df_merged_group_cat.isnull().sum()

##################### MRL_Line grouping done ############################
# Assuming df_merged_group_cat is your DataFrame containing the data
# Group the data based on 'MRL_LINE'
mrl_groups = df_merged_group_cat.groupby('MRL_LINE')

# Now you can perform operations on each group
for mrl_line, group in mrl_groups:
    # mrl_line will contain the unique value of MRL_LINE for the current group
    print(f"MRL_LINE: {mrl_line}")
    print(group)  # 'group' will be a DataFrame containing the data for the current MRL_LINE group

In [ ]:
"""
#### Observation data is not Gausian
"""
##################
#### IQR Upperbound Lower Bound Extreme lower bound and Extreme upper bound
##################
"""
IQR (Interquartile Range), Upper Bound, Lower Bound, Extreme Lower Bound, and Extreme Upper Bound are used to identify and handle outliers in a dataset.

Interquartile Range (IQR):
The IQR is a measure of statistical dispersion that represents the range between the first quartile (25th percentile) and the third quartile (75th percentile) of the data. It is calculated as follows:
IQR = Q3 (75th percentile) - Q1 (25th percentile)

Upper Bound:
The Upper Bound is used to identify outliers on the higher end of the data distribution. It is calculated as follows:
Upper Bound = Q3 (75th percentile) + 1.5 * IQR

Lower Bound:
The Lower Bound is used to identify outliers on the lower end of the data distribution. It is calculated as follows:
Lower Bound = Q1 (25th percentile) - 1.5 * IQR

Extreme Lower Bound:
The Extreme Lower Bound is used to identify potential extreme outliers on the lower end of the data distribution. It is calculated as follows:
Extreme Lower Bound = Q1 (25th percentile) - 3 * IQR

Extreme Upper Bound:
The Extreme Upper Bound is used to identify potential extreme outliers on the higher end of the data distribution. It is calculated as follows:
Extreme Upper Bound = Q3 (75th percentile) + 3 * IQR

Where Q1 is the first quartile (25th percentile) and Q3 is the third quartile (75th percentile) of the data.

In summary, these values help in identifying potential outliers in the data. Data points that fall outside the range defined by the Upper Bound and Lower Bound are considered outliers. Similarly, data points that fall outside the range defined by the Extreme Upper Bound and Extreme Lower Bound are considered potential extreme outliers. The appropriate treatment of outliers depends on the specific analysis and the domain knowledge. Outliers can be removed or transformed to handle their impact on the analysis and model performance.
"""
# Assumption data in months column follows a Gausian distribution we will calculate the boundaries which differentiate the outliers
# Assuming you have the dataset in a DataFrame named 'df_merged_group_cat'

################# MRL_Line - Outliers found for MRL

# Assuming you have the dataset in a DataFrame named 'df_merged_group_cat'

import pandas as pd

# Assuming you have the dataset in a DataFrame named 'df_merged_group_cat'

# Step 1: Select the relevant columns for analysis (Months: 01 to 12)
selected_columns = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
selected_data = df_merged_group_cat[selected_columns]

# Step 2: Create an empty DataFrame to store the results
output_df = pd.DataFrame(columns=["MRL_LINE", "Month", "Upper_Bound_Gaussian", "Lower_Bound_Gaussian", "Mean", "IQR", "Lower_Bridge", "Upper_Bridge", "Extreme_Lower_Bridge", "Extreme_Upper_Bridge"])

# Step 3: Calculate the bounds and statistics for each month and store in the output DataFrame
for mrl_line, group in df_merged_group_cat.groupby('MRL_LINE'):
    # Calculate bounds and statistics for each month in the group
    for month in selected_columns:
        # Select data for the current MRL_LINE and month
        selected_data = group[month]

        # Calculate bounds using Gaussian distribution assumption
        upper_bound_month = selected_data.mean() + 3 * selected_data.std()
        lower_bound_month = selected_data.mean() - 3 * selected_data.std()

        # Calculate IQR and outliers within range
        IQR_month = selected_data.quantile(0.75) - selected_data.quantile(0.25)
        lower_bridge_month = selected_data.quantile(0.25) - (IQR_month * 1.5)
        upper_bridge_month = selected_data.quantile(0.75) + (IQR_month * 1.5)

        # Calculate extreme outliers
        extreme_lower_bridge_month = selected_data.quantile(0.25) - (IQR_month * 3)
        extreme_upper_bridge_month = selected_data.quantile(0.75) + (IQR_month * 3)

        # Append the results to the output DataFrame
        output_df = output_df.append({"MRL_LINE": mrl_line,
                                      "Month": month,
                                      "Upper_Bound_Gaussian": upper_bound_month,
                                      "Lower_Bound_Gaussian": lower_bound_month,
                                      "Mean": selected_data.mean(),
                                      "IQR": IQR_month,
                                      "Lower_Bridge": lower_bridge_month,
                                      "Upper_Bridge": upper_bridge_month,
                                      "Extreme_Lower_Bridge": extreme_lower_bridge_month,
                                      "Extreme_Upper_Bridge": extreme_upper_bridge_month},
                                      ignore_index=True)

# Step 4: Write the output DataFrame to an Excel file
output_file_path = "C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Working_data\\Budget_merged_file_Cat_working_outliers_MRL_Line.xlsx"
output_df.to_excel(output_file_path, index=False)


In [ ]:
import pandas as pd

# Read the data from the Excel file
outliers_data = pd.read_excel("C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Working_data\\Budget_merged_file_Cat_working_outliers_MRL_Line.xlsx")

# Assuming you have the dataset in a DataFrame named 'df_merged_group_cat'

# Create a dictionary to store upper and lower bounds for each month
month_bounds = {}

# Iterate through each month and store the bounds in the dictionary
for month in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]:
    if month in outliers_data['Month'].values:
        upper_bound = outliers_data.loc[outliers_data['Month'] == month, 'Upper_Bound_Gaussian'].values[0]
        lower_bound = outliers_data.loc[outliers_data['Month'] == month, 'Lower_Bound_Gaussian'].values[0]
        month_bounds[month] = {'upper': upper_bound, 'lower': lower_bound}

# Define a function to replace outliers for each month and MRL_LINE
def replace_outliers(row):
    mrl_line = row['MRL_LINE']
    for month in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]:
        if month in df_merged_group_cat.columns and month in month_bounds:
            upper_bound = month_bounds[month]['upper']
            lower_bound = month_bounds[month]['lower']
            value = row[month]

            if mrl_line in set(outliers_data['MRL_LINE']):
                if value > upper_bound:
                    row[month] = upper_bound
                elif value < lower_bound:
                    row[month] = lower_bound

    return row

# Apply the replace_outliers function to the DataFrame
df_merged_group_cat = df_merged_group_cat.apply(replace_outliers, axis=1)

# Write the updated DataFrame to an Excel file
output_file_path = "C:\\Users\\mraj4\\Documents\\OptimumPython\\DS\\DL\\Sunoida\\Working_data\\Budget_merged_file_Cat_with_outliers_MRL_LINE_replaced.xlsx"
df_merged_group_cat.to_excel(output_file_path, index=False)

# Optionally, print the updated DataFrame
print(df_merged_group_cat)